In [ ]:
import tensorflow as tf

import sys
sys.path.insert(0, '../fairml')
import plot
import generate
import models

ctr=1
generate_toys = generate.toys_simple

# models.py

In [ ]:
import tensorflow as tf
import tensorflow.contrib.layers as layers


class Model:

    def __init__(self, name, depth, width):

        self.depth = depth
        self.width = width
        self.name = name


class Classifier(Model):

    def __init__(self, name, depth=3, width=10, n_classes=2):

        super().__init__(name, depth, width)
        self.n_classes = 2

    def build_forward(self, x_in):

        with tf.variable_scope(self.name):

            # input layer
            layer = x_in

            # hidden layers
            for _ in range(self.depth):
                layer = layers.relu(layer, self.width)

            # logits and output
            self.logits = layers.linear(layer, self.n_classes)
            self.output = tf.reshape(layers.softmax(self.logits)[:, 1], shape=(-1, 1))

        self.vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.name)

    def build_loss(self, labels):

        # one hot encode the labels
        one_hot = tf.one_hot(tf.reshape(labels, shape=[-1]), depth=self.n_classes)

        # and build the loss
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=one_hot, logits=self.logits))
        

class Adversary(Model):
    
    def __init__(self, name, depth=1, width=5):
        
        super().__init__(name, depth, width)
        self.loss = None
        self.vars = None
    
    @classmethod
    def create(cls, adv_settings):
        
        adv_type = adv_settings['adv_type']
        
        classes = {'Dummy':DummyAdversary,
                   'GMM':GMMAdversary}
        
        # check if implemented
        if adv_type not in classes:
            raise ValueError('Unknown Adversary type {}.'.format(adv_type))
        
        # return the right one
        adversary = classes[adv_type]
        adv_settings.pop('adv_type')
        return adversary(name=adv_type, **adv_settings)

        
class GMMAdversary(Adversary):
    
    def __init__(self, name, **kwargs):
        
        super().__init__(name, **kwargs)

class DummyAdversary(Adversary):
    
    def __init__(self, name, **kwargs):
        
        super().__init__(name, **kwargs)
        
    def build_loss(self, fX, Z):
        
        with tf.variable_scope(self.name):
            dummy_var = tf.Variable(0.001, name='dummy')
            self.loss = dummy_var**2 # i.e. goes to zero

        self.vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.name)
    
    

# environment.py

In [ ]:
import matplotlib.pyplot as plt


class TFEnvironment:
    
    def __init__(self, generate, name):
        
        # save variables
        self.generate = generate
        self.name = name
        self._xyz = ['X', 'Y', 'Z']
        
        # record losses
        self.loss_history = {'clf':[], 'adv':[], 'comb':[]}
        
        # start the session
        self._start_session()
    
    def _start_session(self):
        
        # start the session
        config=tf.ConfigProto(intra_op_parallelism_threads = 32,
                                 inter_op_parallelism_threads = 32,
                                 allow_soft_placement = True,
                                 device_count = {'CPU': 2})
        
        self.sess = tf.Session(config=config)
    
    def build_graph(self, clf_settings, adv_settings):
        
        print('--- Building computational graphs')
        
        # build the inputs
        batch = self.generate(10)
        self._in = {}
        for xyz in self._xyz:
            tftype = tf.int32 if xyz == 'Y' else tf.float32
            self._in[xyz] = tf.placeholder(tftype, shape=(None, batch[xyz].shape[1]), name='{}_in'.format(xyz))
        
        # build the classifier graph
        self.clf = Classifier(name=self.name+'_clf', **clf_settings)
        self.clf.build_forward(self._in['X'])
        
        # build the adversary graph
        self.adv = Adversary.create(adv_settings)
        
    def build_loss(self):
        
        print('--- Building computational graphs for losses')
        
        # classifier loss
        self.clf.build_loss(self._in['Y'])
        
        # adversary loss
        self.adv.build_loss(self.clf.output, self._in['Z'])
        
    def build_opt(self, lam=1, opt_type='AdamOptimizer', learning_rate=0.05, projection=False):
        
        print('--- Building computational graphs for optimisations')
        
        # optimizer type
        opt = getattr(tf.train, opt_type)
        self.optimizer = opt(learning_rate=learning_rate)
        self.lam = lam
        
        # compute the gradients
        self.grad_clf = self.optimizer.compute_gradients(self.clf.loss, var_list=self.clf.vars)
        self.grad_adv = self.optimizer.compute_gradients(self.adv.loss, var_list=self.adv.vars)
        self.grad_comb = []
        
        # compute the optimisation steps
        self.opt_clf = self.optimizer.apply_gradients(self.grad_clf)
        self.opt_adv = self.optimizer.apply_gradients(self.grad_adv)
        self.opt_comb = None #self.optimizer.apply_gradients(self.grad_comb)
        
    def initialise_variables(self):

        print('--- Initialising TensorFlow variables')

        self.sess.run(tf.global_variables_initializer())
    
    def _get_feed_dict(self, batch):
        
        return {self._in[xyz]:batch[xyz] for xyz in self._xyz}
    
    def pretrain_step(self, batch_size):
        
        # pre-training step (no adversary)
        batch = self.generate(batch_size)
        feed_dict = self._get_feed_dict(batch)
        self.sess.run(self.opt_clf, feed_dict=feed_dict)
        
        # update loss history
        self._write_loss_history(batch)
        
    def train_step_clf(self, batch_size):
        
        pass
    
    def train_step_adv(self, batch_size):
        
        pass
    
    def _write_loss_history(self, batch):
        
        # get current losses
        loss_clf, loss_adv, loss_comb = self._losses(batch)
        
        # append
        self.loss_history['clf'].append(loss_clf)
        self.loss_history['adv'].append(loss_adv)
        self.loss_history['comb'].append(loss_comb)
    
    def _losses(self, batch):
        
        feed_dict = self._get_feed_dict(batch)
        loss_clf, loss_adv = self.sess.run([self.clf.loss, self.adv.loss], feed_dict=feed_dict)
        return loss_clf, loss_adv, loss_clf - self.lam * loss_adv
        
    def predict(self, batch):
        
        feed_dict = self._get_feed_dict(batch)
        preds = self.sess.run(self.clf.output, feed_dict=feed_dict)
        
        return preds
    
    def show_variates(self, batch_size):
        
        print('--- Plot random variates')
        
        # prepare the plot
        batch = self.generate(batch_size)
        fig, ax = plt.subplots(2, 2,
                               figsize=(7,7),
                               gridspec_kw={'height_ratios':[1,4],
                                            'width_ratios':[4,1]},
                               sharex='col',
                               sharey='row')

        # main plot
        plot.variates_main(ax[1,0], batch)
        
        # top plot
        plot.variates_kde(ax[0,0], batch, x_cpt=0)
        
        # right plot
        plot.variates_kde(ax[1,1], batch, x_cpt=1)
    
        # empty axes
        fig.delaxes(ax[0,1])
        fig.show()
    
    def show_performance(self, batch_size):
        
        print('--- Plot classifier performance and fairness')
        
        # get predictions
        batch = {}
        preds = {}
        for z in [None, 1, 0, -1]:
            batch[z] = self.generate(batch_size, z=z)
            preds[z] = self.predict(batch[z])
        
        # prepare the figure
        fig, ax = plt.subplots(2, 2, figsize=(10, 10))
        
        # plot the variates
        plot.variates_main(ax[0,0], batch[None])
        
        # plot the ROC curves
        plot.roc_curves(ax[1,0], batch[1], batch[0], batch[-1], preds[1], preds[0], preds[-1])
        
        # plot the classifier output
        plot.clf_outputs(ax[1,1], preds[1], preds[0], preds[-1])
        
        # plot the decision boundary
        plot.decision_boundary(ax[0,1], batch[None], preds[None])
        
        # show
        fig.show()
        
    def show_loss_history(self):
        
        print('implement me!')
        

In [ ]:
# settings
ctr+=1

clf_settings = {'depth':3, 'width':10}
adv_settings = {'adv_type':'Dummy'}
opt_settings = {'lam':50, 'opt_type':'AdamOptimizer', 'learning_rate':0.05}

# make the environment
tfe = TFEnvironment(generate_toys, 'tf_env_{}'.format(ctr))
tfe.build_graph(clf_settings, adv_settings)
tfe.build_loss()
tfe.build_opt(**opt_settings)
tfe.initialise_variables()

for i in range(50):
    tfe.pretrain_step(10)
#tfe.show_variates(4000)
tfe.show_performance(4000)
tfe.show_loss_history()


